(load-project-yaml-from-git)=
# Load project YAML from Git, Zip, Tar source

After you create your project and you have a project.yaml file with all the necessery metadata within the remote source (Git, zip or gz.tar file), you can simply load that project and run, build, and deploy your functions and workflows.

Run the project automation in {ref}`automate-project-git-source` before you run this workbook.

This notebook presents the steps to load a CI/CD project in MLRun:
1. [Loading a project from a remote URL](#loading-a-project-from-a-remote-url)
2. [Getting a function object](#getting-a-function-object)
3. [Running project functions](#running-project-functions)
5. [Deploying project functions](#deploying-project-functions)
6. [Running the project workflow](#running-the-project-workflow)

Install mlrun using ``pip install mlrun==<mlrun server version>`` or ``sh align_mlrun.sh`` (the default mlrun installer that automatically installs the server version).

In [1]:
import mlrun

## Loading a project from a remote URL

This method can be used for loading an MLRun project from yaml/zip/tar/git/dir or from the MLRun DB.

In [2]:
# project source to load from -'git://url/org/repo.git#<branch-name or refs/heads/.. or refs/tags/..>`.
source = "git://github.com/mlrun/ci-cd-tutorial.git#refs/tags/v3"

**Note -** Add the git branch or refs to the source e.g.: 'git://<url>/org/repo.git#<branch-name or refs/heads/..>'

In [3]:
# load the project
project = mlrun.load_project(
    "./clone", url=source, clone=True, name="my-load-proj", user_project=True
)

For example:
```
# when loading from private repo
project = mlrun.get_or_create_project(name='new-ci-cd-proj',context='./',init_git=True,secrets={"GIT_TOKEN":<github-token>})
# when running functions in a project from a private repo
project.set_secrets({"GIT_TOKEN":<github-token>}
```

See {py:class}`mlrun.projects.load_project`.

In [4]:
# print project yaml
print(project.to_yaml())

## Getting a function object 
Get the function object using the {py:class}`~mlrun.projects.MlrunProject.get_function` method.

This method allows you to get a function object based on the metadata in your project YAML file or from MLRun DB.
````
serving_func = project.get_function('<function name>')
````

In [8]:
serving_func = project.get_function("serving")

In [10]:
serving_func.add_model(
    key="model",
    model_path=train_run.outputs["model"],
    class_name="mlrun.frameworks.sklearn.SklearnModelServer",
)

```{admonition} Tip: Changing the model file path
This serving function points to a model file whose path is stored in the function spec. If you want to change it (for example, to use a newer model file) you need to add the model to the function object and then deploy the function, or alternately, change the function.yaml in the remote source:


      serving_func = project.get_function('serving')
      serving_func.add_model(key='model',model_path=train_run.outputs["model"],
      class_name='mlrun.frameworks.sklearn.SklearnModelServer')
      requirements = ["scikit-learn"]
      serving_dep = project.deploy_function('serving')
```

Test your serving function locally before deploying it.

In [11]:
serving_server = serving_func.to_mock_server()

In [12]:
my_data = """{"inputs":[[-0.60150011,  0.51150308,  0.25701239, -1.51777297, -1.82961288,
         0.22983693, -0.40761625,  0.82325082,  1.1779216 ,  1.08424275,
        -0.7031145 , -0.40608979, -0.36305977,  1.28075006,  0.94445967,
         1.19105828,  1.93498414,  0.69911167,  0.50759757,  0.91565635]]}"""

In [13]:
serving_server.test("/", my_data)

## Running project functions 
Run the function using the {py:class}`~mlrun.projects.MlrunProject.run_function` method both to 
[run jobs locally](./automate-project-git-source.html#running-the-function-locally) 
and, run remotely on the [runtime/cluster](./automate-project-git-source.html#running-the-function-remotely-on-your-cluster). If 
there are any requirements you need to build a new 
image before you run a function. See more details in {ref}`build-function-image`.

In [5]:
project.run_function(
    function="data-fetch", local=True, returns=["train-dataset", "test-dataset"]
)

In [6]:
data_fetch_run = project.run_function(
    function="data-fetch", local=False, returns=["train-dataset", "test-dataset"]
)

In [7]:
train_run = project.run_function(
    function="train",
    inputs={
        "train_data": data_fetch_run.outputs["train-dataset"],
        "test_data": data_fetch_run.outputs["test-dataset"],
    },
)

## Deploying project functions
To deploy a remote function e.g. nuclio or serving function, use the {py:class}`~mlrun.projects.MlrunProject.deploy_function` method. 
You must use this method before invoking Nuclio or serving functions.
````
nuclio_func=project.deploy_function(function='<function name>')

nuclio_func.function.invoke('/',{'int':4})
````

In [14]:
serving_dep = project.deploy_function("serving")

In [15]:
serving_dep.function.invoke("/", my_data)

## Running the project workflow

In [16]:
# run the workflow named main and wait for the pipeline completion (watch=True)
project.run("main", watch=True, engine="remote:kfp")

## Running a scheduled workflow

For more information about scheduling workflows, see {ref}`scheduled-jobs`.

In [17]:
project.run("main", watch=True, schedule="0 * * * *")